In [1]:
#CONNEXION à google drive
import pandas as pd
from google.colab import drive

drive.mount('/Drive')

Mounted at /Drive


In [2]:
#PARAMETRES!!!
isSaveData = False
pathSaveCsv = '/Drive/My Drive'
isRegenerateKeywords = False
isManualDataFrame=False
madescription="voici ma description d'article"
madesignation="voici ma désignation d'article"
import re  
import requests
import io

In [17]:
def creerDataframePourArticleManuel(madescription, madesignation):
  dic={"designation":[madesignation],"description":[madescription],"productid":0,"imageid":None,"prdtypecode":None}
  X=pd.DataFrame(dic)
  return X
  
#CHARGEMENT des fichiers
if (not isManualDataFrame):
  url = "https://raw.githubusercontent.com/JulienJ-44/rakuteam/main/Datasets/Dataset_challenge.csv"
  download = requests.get(url).content
  X=pd.read_csv(io.StringIO(download.decode('utf-8')), index_col=0)
else:
  #OU création d'un dataframe
  X=creerDataframePourArticleManuel(madescription, madesignation)

In [18]:
X.head()

,designation,description,productid,imageid,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10.0
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280.0
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50.0
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280.0
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705.0


In [19]:

#y = pd.read_csv("/Drive/My Drive/Colab/Y_train_CVw08PX.csv", index_col=0) 

#stockage de tous les codes de classe dans une liste
#classes_codes = (y['prdtypecode'].value_counts().index.tolist())

#PREPARATION: FUSION de X et y et remplacement des VALEURS MANQUANTES par ""
df = X.drop(['imageid'], axis = 1)#
#on joint X et y
#df = pd.merge(df1, y, left_index=True, right_index=True)

#valeurs MANQUANTES
df['description']=df['description'].fillna("")
df['designation']=df['designation'].fillna("")
#turn all text to LOWER case
df['description'] = df['description'].str.lower()
df['designation'] = df['designation'].str.lower()

#recodage des caractères HTML
df.replace(["&eacute;","&egrave;","&euml;","&ecirc;","&ocirc;","&ugrave;","&ccedil;","&agrave;","&nbsp;","&amp;","&#39;","'",'n°']
           , ["é","è","ë","ê","ô","ù","ç","à"," ","&"," "," ",'numéro '], inplace=True, regex=True)

"""
Return the text in lower case after removing html tags
"""    
def pre_process(text):
    # lowercase
    #text=text.lower()
    #remove tags
    text=re.sub('<.*?>'," ",text) 
    return text

df['description'] = df['description'].apply(lambda x:pre_process(x))
df['designation'] = df['designation'].apply(lambda x:pre_process(x))

In [20]:
#FONCTION pour supprimer les mots inutiles
# Download the stopwords
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk as nltk
#utiliser @st.cache !!! pour STREAMLIT
nltk.download('punkt') #télécharge les paquets language (dont FR)
nltk.download('stopwords')

#Ajoute des stopwords à la liste française prédéfinie
stop_words = set(stopwords.words('french'))
stop_words.update(stopwords.words('english'))
stop_words.update(stopwords.words('german'))
stop_words.update(['-','de','en','pour','la','(',')','the','/li','/strong','strong','and','non','eacute','comme','cette','of'])

def filtrer_mots_inutiles(texte):
  """
  Fonction pour transformer un texte en une suite de mots séparés par des espaces
  et en excluant les stopwords et les mots de moins de 2 caractères
  """
  mots = word_tokenize(texte, language='french')
  tokens = []
  for mot in mots:
    if (mot not in stop_words and len(mot)>1):
      tokens.append(mot.lower())
  mastring=" ".join(tokens)
  return mastring

df['designation']= df['designation'].apply(lambda x: filtrer_mots_inutiles(x))
df['description']= df['description'].apply(lambda x: filtrer_mots_inutiles(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
if (isSaveData):
  df.to_csv(f'{pathSaveCsv}/dataset_challenge_cleaned.csv')

In [22]:
#DEBUT PREPROCESSING
if (isSaveData):
  df=pd.read_csv(f'{pathSaveCsv}/dataset_challenge_cleaned.csv')
df['designation']=df['designation'].fillna("")
#valeurs MANQUANTES
df['description']=df['description'].fillna("")

In [23]:
#Tokenisation (Séparation du texte en phrases et mots) du champ "designation"

#en phrases
tokenizer = PunktSentenceTokenizer()
df['desi_nb_phrases']= df['designation'].apply(lambda x: len(tokenizer.tokenize(str(x))))
df['desc_nb_phrases']= df['description'].apply(lambda x: len(tokenizer.tokenize(str(x))))
#en mots
#df['desi_nb_mots+']= df['designation'].apply(lambda x: len(x.split(" "))
#df['desc_nb_mots+']= df['description'].apply(lambda x: len(x.split(" "))

In [24]:
#IDF ("inverse de fréquence de document")
#formule: log(nbr d articles / nb d articles dans lequels le terme apparaît)
from sklearn.feature_extraction.text import TfidfVectorizer
if (isRegenerateKeywords):
  #transformation de la colonne en type liste
  words_desi=df['designation'].tolist()
  #words_desc=df['description'].tolist()

  # create object 
  vect  = TfidfVectorizer()  
  # get tf-df values 
  result = vect.fit_transform(words_desi) 
  # get idf values 
  #sorted_vocabulary = dict(sorted(vect.vocabulary_.items(), key=lambda item: item[0], reverse=True))
  #print(vect.vocabulary_)
  #print(vect.get_feature_names())
  #print(vect.idf_)

In [25]:
if (isRegenerateKeywords):
  #fonctions de manipulation des TFIDF
  def sort_coo(coo_matrix):
      tuples = zip(coo_matrix.col, coo_matrix.data)
      return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

  def extract_topn_from_vector(feature_names, sorted_items, topn=10):
      """get the feature names and tf-idf score of top n items"""
      
      #use only topn items from vector
      sorted_items = sorted_items[:topn]

      score_vals = []
      feature_vals = []
      
      # word index and corresponding tf-idf score
      for idx, score in sorted_items:
          
          #keep track of feature name and its corresponding score
          score_vals.append(round(score, 3))
          feature_vals.append(feature_names[idx])

      #create a tuples of feature,score
      #results = zip(feature_vals,score_vals)
      results= {}
      for idx in range(len(feature_vals)):
          results[feature_vals[idx]]=score_vals[idx]
      
      return results

In [26]:
if (isRegenerateKeywords):
  #TF fréquence du terme dans l ensbl de la categorie d articles
  feature_names=vect.get_feature_names()
  #on étudie le champ designation ( à remplacer par "description" si besoin)
  colonneClass = "designation" 
  class_text_cols = []
  #on parcourt la liste des codes de classes
  lst_keywords_byclass={}
  classes_codes = (df['prdtypecode'].value_counts().index.tolist())
  #classes_codes=[2583, 1220, 22240, ...]
  for class_code in classes_codes:
    #on copie dans un df les lignes correspondant au code de classe actuel
    df_class = df[df["prdtypecode"]==class_code]
    #on joint tous les élements de la colonne dans une chaîne
    class_text = ' '.join(df_class[colonneClass])
    #affichage des infos de la classe
    print("Classe:", class_code, "\n", len(df_class.index), " éléments")
    #tfidf
    class_text_encode = vect.transform([class_text])

    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(class_text_encode.tocoo())
    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,15)
    # now print the results
    print("===Keywords===")
    for k in keywords:
        print(k,keywords[k])
    lst_keywords_byclass[class_code]=keywords
    print("\n")

  #lst_keywords_byclass contient: {1180:[mot1 mot2 mot3 ...], 2520:[mot1 mot2 mot3 ...] ...}

In [27]:
#SAUVEGARDES des 15 keyword par classe
import pickle
if (isSaveData):
  dict = lst_keywords_byclass
  f = open(f'{pathSaveCsv}/dico_keywords_tfidf_15.pkl',"wb")
  pickle.dump(dict,f)
  f.close()

In [28]:
if (not isSaveData):
  print(pathSaveCsv)
  # reading the dictionnary des 15 keyword
  with open(f'{pathSaveCsv}/dico_keywords_tfidf_15.pkl', 'rb') as handle: 
      data = handle.read() 
  print("Data type before reconstruction : ", type(data)) 
  # reconstructing the data as dictionary 
  lst_keywords_byclass = pickle.loads(data) 
  print("Data type after reconstruction : ", type(lst_keywords_byclass)) 
print(lst_keywords_byclass) 

/Drive/My Drive
Data type before reconstruction :  <class 'bytes'>
Data type after reconstruction :  <class 'dict'>
{2583.0: {'piscine': 0.58, 'kit': 0.288, 'bâche': 0.243, 'spa': 0.194, 'pompe': 0.161, 'bulles': 0.154, 'intex': 0.148, 'cm': 0.137, 'ronde': 0.123, 'acier': 0.12, 'gonflable': 0.12, 'filtration': 0.111, 'gré': 0.111, 'filtre': 0.106, 'bois': 0.106}, 1560.0: {'cm': 0.271, 'matelas': 0.224, 'cuisine': 0.222, 'table': 0.172, 'chaise': 0.164, 'blanc': 0.157, 'bois': 0.15, 'tissu': 0.146, 'rangement': 0.141, 'salle': 0.128, 'fournitures': 0.123, 'design': 0.115, 'sommier': 0.115, 'gris': 0.114, 'pieds': 0.113}, 1300.0: {'générique': 0.603, 'rc': 0.338, 'drone': 0.312, 'dji': 0.241, 'quadcopter': 0.157, 'mavic': 0.143, 'avion': 0.128, 'fpv': 0.127, 'caméra': 0.119, 'maquette': 0.118, 'voiture': 0.112, 'wifi': 0.108, 'car': 0.089, 'pro': 0.089, 'batterie': 0.085}, 2060.0: {'led': 0.293, 'noël': 0.26, 'décoration': 0.229, 'lampe': 0.228, 'décor': 0.204, 'bricolage': 0.171, 'pein

In [29]:
#TEMP df_sample!!!
#df_sample=df.head(1000) #TEMP!!!!
#la liste des keywords stockés par classe a été limitée à 10, faut il aller plus loin, performances?
"""
Fontion de scoring qui renvoie un score selon le nombre d'occurences des mots et leur score idf
texte: texte à lire mot à mot pour scorer les mots
dict_keywords_idf:dictionnaire des keywords/idf d'une des classes produits
"""
import re
def scoring(texte, dict_keywords_idf):
#texte = texte de la désignaton/description de l'article
#dict_keywords_idf = [mot1 mot2 mot3 ...]
  #print(texte)
  #print(dict_keywords_idf)
  score = 0.0
  for keyword_key,keyword_idf in dict_keywords_idf.items():
    nb_occur = len(re.findall(keyword_key, texte, flags=re.IGNORECASE))
    score += nb_occur*keyword_idf
  #print(score)
  return score

df["best_idf"]=0
#lst_keywords_byclass contient: {1180:[mot1 mot2 mot3 ...], 2520:[mot1 mot2 mot3 ...] ...}
#lst_keywords_byclass = sorted(lst_keywords_byclass)
for (key_kw, value_kw) in sorted(lst_keywords_byclass.items()):
  #key_kw=1180
  #value_kw=[mot1 mot2 mot3 ...]
  df['class_'+str(key_kw)]=df['designation'].apply(lambda x: scoring(x, value_kw))

In [30]:
#df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
df.head()

,designation,description,productid,prdtypecode,desi_nb_phrases,desc_nb_phrases,best_idf,class_10.0,class_40.0,class_50.0,class_60.0,class_1140.0,class_1160.0,class_1180.0,class_1280.0,class_1281.0,class_1300.0,class_1301.0,class_1302.0,class_1320.0,class_1560.0,class_1920.0,class_1940.0,class_2060.0,class_2220.0,class_2280.0,class_2403.0,class_2462.0,class_2522.0,class_2582.0,class_2583.0,class_2585.0,class_2705.0,class_2905.0
0,olivia personalisiertes notizbuch 150 seiten p...,,3804725264,10.0,1,0,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.157,0.115,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.190,0.0,0.0,0.115,0.000,0.021
1,journal arts numéro 133 28/09/2001 art marche ...,,436067568,2280.0,1,0,0,0.518,0.0,0.00,0.0,0.0,0.265,0.0,0.176,0.000,0.338,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,1.301,0.235,0.000,0.272,0.0,0.0,0.000,0.000,0.000
2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,201115110,50.0,1,1,0,0.000,0.1,0.48,0.0,0.0,0.000,0.0,0.000,0.275,0.000,0.0,0.0,0.144,0.000,0.0,0.26,0.0,0.0,0.000,0.625,0.657,0.297,0.0,0.0,0.000,0.000,0.020
3,peluche donald europe disneyland 2000 marionne...,,50418756,1280.0,1,0,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.222,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000
4,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,278535884,2705.0,1,1,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.111,0.000


In [31]:
col_start=7
df["best_idf"]=df.iloc[:,col_start:].idxmax(axis=1)
#df["best_idf"]= [2583 if (s.sum()==0) else df.iloc[:,col_start:].idxmax(axis=1) for s in df.iloc[:,1col_start1:]]
#gestion des cas où toutes les colonnes valent 0
#on affecte la classe 2583 (majoritaire)
for i in range(len(df)):
  sum = df.iloc[i,col_start:].sum()
  if (sum == 0):
    df["best_idf"][i]="2583"

print("#Articles:", len(df))
print(df["best_idf"].value_counts())
#path = '/Drive/My Drive/Projet Rakuten'
#df.to_csv(f'{path}/damien.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


#Articles: 98728
class_2583.0    8554
class_2280.0    8197
class_1300.0    7174
class_1160.0    6611
2583            6356
class_2403.0    5685
class_2060.0    4546
class_1920.0    4521
class_1560.0    4282
class_2582.0    4107
class_1320.0    3968
class_1280.0    3310
class_2522.0    3233
class_2905.0    2728
class_2585.0    2623
class_1140.0    2550
class_60.0      2520
class_40.0      2391
class_1940.0    2090
class_2220.0    1920
class_2705.0    1718
class_10.0      1636
class_50.0      1539
class_1281.0    1527
class_1301.0    1458
class_1302.0    1433
class_2462.0    1258
class_1180.0     793
Name: best_idf, dtype: int64


In [32]:
df.head()

,designation,description,productid,prdtypecode,desi_nb_phrases,desc_nb_phrases,best_idf,class_10.0,class_40.0,class_50.0,class_60.0,class_1140.0,class_1160.0,class_1180.0,class_1280.0,class_1281.0,class_1300.0,class_1301.0,class_1302.0,class_1320.0,class_1560.0,class_1920.0,class_1940.0,class_2060.0,class_2220.0,class_2280.0,class_2403.0,class_2462.0,class_2522.0,class_2582.0,class_2583.0,class_2585.0,class_2705.0,class_2905.0
0,olivia personalisiertes notizbuch 150 seiten p...,,3804725264,10.0,1,0,class_2522.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.157,0.115,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.190,0.0,0.0,0.115,0.000,0.021
1,journal arts numéro 133 28/09/2001 art marche ...,,436067568,2280.0,1,0,class_2280.0,0.518,0.0,0.00,0.0,0.0,0.265,0.0,0.176,0.000,0.338,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,1.301,0.235,0.000,0.272,0.0,0.0,0.000,0.000,0.000
2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,201115110,50.0,1,1,class_2462.0,0.000,0.1,0.48,0.0,0.0,0.000,0.0,0.000,0.275,0.000,0.0,0.0,0.144,0.000,0.0,0.26,0.0,0.0,0.000,0.625,0.657,0.297,0.0,0.0,0.000,0.000,0.020
3,peluche donald europe disneyland 2000 marionne...,,50418756,1280.0,1,0,class_1280.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.222,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000
4,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,278535884,2705.0,1,1,class_2705.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.111,0.000


In [33]:
#Expressions régulières pour identifier les différentes unités
#TODO transformer en numérique pas en liste
import re

#nombre de nombres à 2 chiffres ou +
r = re.compile("[0-9]{2,}") 
df['desi_nb2chiffres+']= df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_nb2chiffres+']= df['description'].apply(lambda x: min(1,len(r.findall(x))))
#XGo ou XMo ou XTo
#r = re.compile("([\d.]+)\s?(go|mo|to|Go|Mo|To|giga|gigas)") 
#df['desi_Go']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
#df['desc_Go']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#N°X
r = re.compile("(numéro )") 
df['desi_num']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desi_num']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
r = re.compile("[Nn][°]\s?[\d]+") 
#df['desi_num']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_num']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Poids kg Kg mg g
r = re.compile("[0-9\.]+[kKm]?[g]") #([\d.]+)\s+(lbs?|oz|g|kg) 
r = re.compile("([\d.]+)\s?(KG|Kg|g|kg|mg)[\s.]") #(nombres ou .)+ / (espace)+ / (kg mg ..)
df['desi_poids']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_poids']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Taille cm mm m
r = re.compile("([\d.]+)\s?(cm|mm|m|M)[\s.]") 
df['desi_long']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_long']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Volume mL cL dL
r = re.compile("([\d.]+)\s?(mL|L|ml|l|cl)[\s.]") 
df['desi_vol']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_vol']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Age
r = re.compile("([\d.]+)\s?(an|ans|An|Ans|mois|Mois)[\s.]") 
df['desi_ans_mois']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_ans_mois']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Pièces
r = re.compile("([\d.]+)\s?(pc|pcs|pièces|pièce)[\s.]") 
df['desi_pieces']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_pieces']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#%
#r = re.compile("([\d.]+)\s?%") 
#df['desi_pourcent']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
#df['desc_pourcent']=df['description'].apply(lambda x: min(1,len(r.findall(x))))

In [36]:
import re
def feature(df) :
    df['desi_word_count'] = df['designation'].apply(lambda x : len(str(x).split()))
    df['desi_char_count (w/o space)'] = df['designation'].apply(lambda x : len(x.replace(" ","")))
    #df['desi_word_density'] = df['desi_word_count'] / (df['desi_char_count'] + 1)
    df['desi_total_length'] = df['designation'].apply(len)
    #df['desi_capitals'] = df['designation'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    # df['desi_caps_vs_length'] = df.apply(lambda row: float(row['desi_capitals'])/float(row['desi_total_length']),axis=1)
    df['desi_num_exclamation_marks'] =df['designation'].apply(lambda x: x.count('!'))
    df['desi_num_question_marks'] = df['designation'].apply(lambda x: x.count('?'))
    #df['desi_num_punctuation'] = df['designation'].apply(lambda x: sum(x.count(w) for w in '.,;:'))
    #df['desi_num_symbols'] = df['designation'].apply(lambda x: sum(str(x).count(w) for w in '*&$%+-/'))
    df['desi_num_unique_words'] = df['designation'].apply(lambda x: len(set(w for w in x.split())))
    df['desi_words_vs_unique'] = df['desi_num_unique_words'] / df['desi_word_count']
    df["desi_word_unique_percent"] =  df["desi_num_unique_words"]*100/df['desi_word_count']
    
    df['descri_word_count'] = df['description'].apply(lambda x : len(str(x).split()))
    df['descri_char_count (w/o space)'] = df['description'].apply(lambda x : len(str(x).replace(" ","")))
    #df['descri_word_density'] = df['descri_word_count'] / (df['descri_char_count'] + 1)
    df['descri_total_length'] = df['description'].apply(lambda x :len(str(x)))
   # df['descri_capitals'] = df['description'].apply(lambda comment: sum(1 for c in str(comment) if c.isupper()))
    # df['descri_caps_vs_length'] = df.apply(lambda row: float(row['descri_capitals'])/float(row['descri_total_length']),axis=1)
    df['descri_num_exclamation_marks'] =df['description'].apply(lambda x: str(x).count('!'))
    df['descri_num_question_marks'] = df['description'].apply(lambda x: str(x).count('?'))
    #df['descri_num_punctuation'] = df['description'].apply(lambda x: sum(str(x).count(w) for w in '.,;:'))
   # df['descri_num_symbols'] = df['description'].apply(lambda x: sum(str(x).count(w) for w in '*&$%'))
    df['descri_num_unique_words'] = df['description'].apply(lambda x: len(set(w for w in str(x).split())))
    df['descri_words_vs_unique'] = df['descri_num_unique_words'] / df['descri_word_count']
    df["descri_word_unique_percent"] =  df["descri_num_unique_words"]*100/df['descri_word_count']
    return df

feature(df)

,designation,description,productid,prdtypecode,desi_nb_phrases,desc_nb_phrases,best_idf,class_10.0,class_40.0,class_50.0,class_60.0,class_1140.0,class_1160.0,class_1180.0,class_1280.0,class_1281.0,class_1300.0,class_1301.0,class_1302.0,class_1320.0,class_1560.0,class_1920.0,class_1940.0,class_2060.0,class_2220.0,class_2280.0,class_2403.0,class_2462.0,class_2522.0,class_2582.0,class_2583.0,class_2585.0,class_2705.0,class_2905.0,desi_nb2chiffres+,desc_nb2chiffres+,desi_num,desc_num,desi_poids,desc_poids,desi_long,desc_long,desi_vol,desc_vol,desi_ans_mois,desc_ans_mois,desi_pieces,desc_pieces,desi_word_count,desi_char_count (w/o space),desi_total_length,desi_num_exclamation_marks,desi_num_question_marks,desi_num_unique_words,desi_words_vs_unique,desi_word_unique_percent,descri_word_count,descri_char_count (w/o space),descri_total_length,descri_num_exclamation_marks,descri_num_question_marks,descri_num_unique_words,descri_words_vs_unique,descri_word_unique_percent
0,olivia personalisiertes notizbuch 150 seiten p...,,3804725264,10.0,1,0,class_2522.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.157,0.115,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.190,0.000,0.000,0.115,0.000,0.021,1,0,0,0,0,0,0,0,0,0,0,0,0,0,10,70,79,0,0,10,1.000000,100.000000,0,0,0,0,0,0,NaN,NaN
1,journal arts numéro 133 28/09/2001 art marche ...,,436067568,2280.0,1,0,class_2280.0,0.518,0.0,0.000,0.000,0.0,0.265,0.000,0.176,0.000,0.338,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,1.301,0.235,0.000,0.272,0.000,0.000,0.000,0.000,0.000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,23,134,156,0,0,22,0.956522,95.652174,0,0,0,0,0,0,NaN,NaN
2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,201115110,50.0,1,1,class_2462.0,0.000,0.1,0.480,0.000,0.0,0.000,0.000,0.000,0.275,0.000,0.000,0.0,0.144,0.000,0.000,0.26,0.000,0.000,0.000,0.625,0.657,0.297,0.000,0.000,0.000,0.000,0.020,0,1,0,0,0,0,0,0,0,0,0,0,0,0,10,63,72,0,0,10,1.000000,100.000000,70,469,538,0,0,53,0.757143,75.714286
3,peluche donald europe disneyland 2000 marionne...,,50418756,1280.0,1,0,class_1280.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.222,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7,49,55,0,0,7,1.000000,100.000000,0,0,0,0,0,0,NaN,NaN
4,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,278535884,2705.0,1,1,class_2705.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.111,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,12,13,0,0,2,1.000000,100.000000,19,111,129,0,0,19,1.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98723,robe soutien sac soins suspendre vêtements pro...,robe porte-soins sac protecteur vêtements hang...,3959381879,NaN,1,1,class_1320.0,0.000,0.0,0.148,0.117,0.0,0.000,0.000,0.000,0.000,0.089,0.000,0.0,0.363,0.000,0.000,0.00,0.000,0.224,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,1,0,0,0,0,0,1,0,0,0,0,0,0,11,69,79,0,0,11,1.000000,100.000000,86,594,679,0,0,73,0.848837,84.883721
98724,livre phrases français thaïlandais,,1546314127,NaN,1,0,class_2705.0,0.000,0.0,0.000,0.000,0.0,0.130,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.088,0.000,0.000,0.000,0.000,0.000,0.167,0.021,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,31,34,0,0,4,1.000000,100.000000,0,0,0,0,0,0,NaN,NaN
98725,tifon1 200 espa pompe piscine,tension monophasé 230 puissance cv type pompe ...,2887191300,NaN,1,1,class_2583.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.935,0.000,0.000,0.000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,5,25,29,0,0,5,1.000000,100.000000,17,11

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98728 entries, 0 to 98727
Data columns (total 64 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   designation                    98728 non-null  object 
 1   description                    98728 non-null  object 
 2   productid                      98728 non-null  int64  
 3   prdtypecode                    84916 non-null  float64
 4   desi_nb_phrases                98728 non-null  int64  
 5   desc_nb_phrases                98728 non-null  int64  
 6   best_idf                       98728 non-null  object 
 7   class_10.0                     98728 non-null  float64
 8   class_40.0                     98728 non-null  float64
 9   class_50.0                     98728 non-null  float64
 10  class_60.0                     98728 non-null  float64
 11  class_1140.0                   98728 non-null  float64
 12  class_1160.0                   98728 non-null 

In [38]:
if (isSaveData):
  df.to_csv(f'{pathSaveCsv}/features_texte.csv')